In [ ]:
import os
import sys
from pathlib import Path
sys.path.insert(0,"c:\\Users\\lenovo\\Documents\\GitHub\\o-kidia")
from src.data_manipulation.game_session.game_session import GameSession
from src.data_manipulation.game_session.challenge import CrocosMazeChallenge
import pandas as pd

In [ ]:
##sys.path.insert(0,"c:\\Users\\lenovo\\Documents\\GitHub\\o-kidia")
game_session = GameSession.from_json(os.path.join("..","tests","game_session","dummy_data","logs","1.json"))

In [ ]:
digit_inputs = game_session.to_dataframe().sort_values(by=["ts"], ascending=True)

In [ ]:
digit_inputs[digit_inputs["ts"] > 115]

In [ ]:
import seaborn as sns
def plot_challenge(game_session: GameSession, challenge: int, ax=None):
    curve = pd.DataFrame.from_records([{"x": point[0], "y": point[1], "t": point[2], "type": "model"} for point in game_session.sorted_activities[0].challenges[challenge].curve_points()])
    curve_user = pd.DataFrame.from_records([{"x": point[0], "y": point[1], "t": point[2], "type": "user"} for point in game_session.sorted_activities[0].challenges[challenge].digit_curve()])
    sns.lineplot(x="x", y="y", data=curve, color="red", ax=ax, sort=False)
    sns.lineplot(x="x", y="y", data=curve_user, ax=ax, sort=False)

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(5, 3)
fig.set_size_inches(20, 20)
for challenge in range(len(game_session.sorted_activities[0].challenges)):
    plot_challenge(game_session, challenge, axis[challenge // 3, challenge % 3])


In [ ]:
import numpy as np
plt.figure(figsize=(10, 10))
def time_per_sec(x: int):
    return np.sqrt(x)

sns.scatterplot(x=range(30), y=list(map(time_per_sec, range(30))))
plt.xticks(range(30), list(map(str, range(30))))

In [ ]:
import plotly.express as px
def plot_3d(game_session: GameSession, challenge: int):
    curve = pd.DataFrame.from_records([{"x": point[0], "y": point[1], "t": point[2], "type": "model"} for point in game_session.sorted_activities[0].challenges[challenge].curve_points()])
    curve_user = pd.DataFrame.from_records([{"x": point[0], "y": point[1], "t": point[2], "type": "user"} for point in game_session.sorted_activities[0].challenges[challenge].digit_curve()])
    curve_df = pd.concat([curve, curve_user])
    fig = px.line_3d(curve_df, x="t", y="x", z="y", color="type")
    fig.show()

In [ ]:
plot_3d(game_session, 0)

In [ ]:
plot_3d(game_session, 3)

In [ ]:
plot_3d(game_session, 12)

In [ ]:
plot_3d(game_session, 13)

In [ ]:
from tslearn.metrics import lcss, dtw ## longest common subsequence et dynamic time wraping

In [ ]:
[lcss(
    np.asarray([ (point[0], point[1]) for point in challenge.digit_curve()]),
    np.asarray([ (point[0], point[1]) for point in challenge.curve_points()]),
    global_constraint="itakura",
    itakura_max_slope=4
) for challenge in game_session.sorted_activities[0].challenges]

In [ ]:

[1/(1 + dtw(
    np.asarray([ (point[0], point[1]) for point in challenge.digit_curve()]),
    np.asarray([ (point[0], point[1]) for point in challenge.curve_points()])
)) for challenge in game_session.sorted_activities[0].challenges]

In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 24/03/2022
#Creation of folders for each user
import os
for i  in range (1,71):
    os.mkdir("Enfant_{}.".format(i))

In [ ]:

#Test code by Nadim ABOU CHROUCH
#Date : 24/03/2022
#Test pour la creation des HeatMaps en utilisant np.histogram
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter
def plot_HeatMap(game_session: GameSession, challenge: int, enfant: int):
    curve = pd.DataFrame.from_records([{"x_model": point[0], "y_model": point[1]} for point in game_session.sorted_activities[0].challenges[challenge].curve_points()])
    curve_user = pd.DataFrame.from_records([{"x_user": point[0], "y_user": point[1]} for point in game_session.sorted_activities[0].challenges[challenge].digit_curve()])
    # print (curve)
    # print (curve_user)

    data_model = curve.to_numpy();
    data_user = curve_user.to_numpy();
    x_model = np.zeros(len(data_model))
    y_model = np.zeros(len(data_model))
    x_user = np.zeros(len(data_user))
    y_user = np.zeros(len(data_user))
    
    for i in range(0,len(data_model)) :
        x_model[i] = data_model[i][0]
        y_model[i] = data_model[i][1]
    #heatmap for model
    heatmap,xedges,yedges = np.histogram2d(x_model,y_model,bins=(200,200))
    heatmap = gaussian_filter(heatmap, sigma=8)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.clf()
    plt.imshow(heatmap.T, extent=extent, origin='lower',cmap=cm.jet)
    plt.title("Model")
    plt.savefig(os.path.join('Enfant_{}'.format(enfant), 'HeatMapModel_{}'.format(challenge))) 
    plt.show()
    for i in range(0,len(data_user)) :
        x_user[i] = data_user[i][0]
        y_user[i] = data_user[i][1]
    #heatmap for user
    heatmap,xedges,yedges = np.histogram2d(x_user,y_user,bins=(200,200))
    heatmap = gaussian_filter(heatmap, sigma=8)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.clf()
    plt.imshow(heatmap.T, extent=extent, origin='lower',cmap=cm.jet)
    plt.title("User")
    plt.savefig(os.path.join('Enfant_{}'.format(enfant), 'HeatMapUser_{}'.format(challenge))) 
    plt.show()
    


In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 24/03/2022
#Test function plot_HeatMap

x = plot_HeatMap(game_session,12,1)

In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 25/03/2022
#Creation of function that generate heatmaps and save them
def generate_HeatMaps(game_session: GameSession, enfant: int):
    for i in range(0,14):
        plot_HeatMap(game_session,i,enfant)

In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 25/03/2022
#Test generate_HeatMaps
generate_HeatMaps(game_session,1)

In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 04/04/2022

#Creating Heatmaps for each child
for i in range (59,61):
    game_session = GameSession.from_json(os.path.join("..","tests","game_session","dummy_data","logs","{}.json".format(i)))
    generate_HeatMaps(game_session,i)
    

In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 25/03/2022
#Studying the Heatmaps

from PIL import Image
import matplotlib.image as image

#Extracting pixels
im = Image.open('HeatMapUser_1.png','r')
pixel_value = list(im.getdata())
#print(pixel_value)
pixel_value_flat = [x for sets in pixel_value for x in sets]
#print(pixel_value_flat)
print (len(pixel_value_flat))

#Converting the image into a matrix
img=image.imread('HeatMapUser_1.png')
print('The Shape of the image is:',img.shape)
ar = np.array(img)
print('The image as array is:')
print(ar)


In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 28/03/2022
#VGG 19
#VGG using keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

model = VGG19()
    # load an image from file
image = load_img('HeatMapUser_12.png', target_size=(224, 224))
    # convert the image pixels to a numpy array
image = img_to_array(image)
    # reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
image = preprocess_input(image)

    # load an image from file
image1 = load_img('HeatMapModel_12.png', target_size=(224, 224))
    # convert the image pixels to a numpy array
image1 = img_to_array(image1)
    # reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
    # prepare the image for the VGG model
image1 = preprocess_input(image1)

In [ ]:


#Test code by Nadim ABOU CHROUCH
#Date : 28/03/2022
#VGG 19
# predict the probability across all output classes
yhat = model.predict(image)
yhat1 = model.predict(image1)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))
print (yhat[0][0:10])
print (yhat1[0][0:10])
# Distance Calculation
dist = np.linalg.norm(yhat[0]-yhat1[0])
print(dist)



In [ ]:
#Test code by Nadim ABOU CHROUCH
#Date : 28/03/2022
#Converting 1000 features to 100
def FeaturesReduction (y):
    yhat_red = np.zeros(100)
    #print (yhat_red)
    i=0;
    for j in range(0,100):
        yhat_red[j] = sum(y[0][i:i+10])
        i += 10
    yhat_red = yhat_red/10
    #print(yhat_red)
    #print (len(yhat_red))
    return yhat_red


In [ ]:

#Test code by Nadim ABOU CHROUCH
#Date : 28/03/2022
#Test FeaturesReduction
yhat_red = FeaturesReduction(yhat)
yhat1_red = FeaturesReduction(yhat1)
print(yhat_red)
print(yhat1_red)
# Distance Calculation
dist = np.linalg.norm(yhat_red-yhat1_red)
print(dist)

In [ ]:
#len(list(game_session.sorted_activities[0].challenges[10].digit_curve()))
len(list(game_session.sorted_activities[0].challenges[10].curve_points()))

# Test DTW

In [ ]:
from tslearn.metrics import dtw_path

In [ ]:
path_sims = [dtw_path(
    np.asarray([ (point[0], point[1]) for point in challenge.digit_curve()]),
    np.asarray([ (point[0], point[1]) for point in challenge.curve_points()])
) for challenge in game_session.sorted_activities[0].challenges]

In [ ]:
path_sims

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
from typing import Tuple
def dtw_plot(path_sim: Tuple[list, int], digit_points: np.array, curve_points: np.array, ax=plt):
    path, sim = path_sim
    ax.plot(digit_points[:, 0], digit_points[:, 1], "g-", label='User digits trajectory')
    ax.plot(curve_points[:, 0], curve_points[:, 1], "b-", label='Optimal trajectory')

    for positions in path:
        ax.plot(
            [
                digit_points[positions[0], 0],
                curve_points[positions[1], 0]
            ],
            [
                digit_points[positions[0], 1],
                curve_points[positions[1], 1]
            ],
            color='orange',
        )

    ax.legend()
    if (isinstance(ax, plt.Axes)):
        ax.set_title(f"DTW distance: {(1/(1+sim)):.2f}")


In [ ]:
dtw_plot(path_sims[0], np.asarray([ (point[0], point[1]) for point in game_session.sorted_activities[0].challenges[0].digit_curve()]), np.asarray([ (point[0], point[1]) for point in game_session.sorted_activities[0].challenges[0].curve_points()]))

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(5, 3)
fig.set_size_inches(20, 20)
fig.tight_layout()
for i, challenge in enumerate(game_session.sorted_activities[0].challenges):
    dtw_plot(path_sims[i], np.asarray([ (point[0], point[1]) for point in challenge.digit_curve()]), np.asarray([ (point[0], point[1]) for point in challenge.curve_points()]), axis[i // 3, i % 3])


In [ ]:
indicateurs = [
    ("Axel","Attention", "Educateur", 2),
    ("Axel","Attention", "Jeu", 1),
    ("Axel","Mémoire","Educateur", 1),
    ("Axel","Mémoire","Jeu", 1),
    ("Axel","Langage","Educateur", 2),
    ("Axel","Langage","Jeu", 3),
    ("Axel","Fonctions Executives","Educateur",1),
    ("Axel","Fonctions Executives","Jeu",2),
    ("Axel","Capacités visuo-spatiales","Educateur",2),
    ("Axel","Capacités visuo-spatiales","Jeu",1),
    ("Axel","Résolution pb","Educateur",3),
    ("Axel","Résolution pb","Jeu",2),
    ("Axel","Planification","Educateur",4),
    ("Axel","Planification","Jeu",1),
    ("Axel","Habilités sociales","Educateur",1),
    ("Axel","Habilités sociales","Jeu",2),
    ("Axel","Motricité","Educateur",1),
    ("Axel","Motricité","Jeu",2),
    ("Axel","Emotion","Educateur",2),
    ("Axel","Meta-cognition","Educateur",1),
]

indicateurs = pd.DataFrame(indicateurs, columns=["Enfant","Indicateur", "Type", "Valeur"])
indicateurs

In [ ]:
import plotly.express as px
fig = px.sunburst(indicateurs, path=['Enfant', 'Type', 'Indicateur'], values='Valeur')
fig.show()

In [ ]:
fig = px.bar_polar(indicateurs, r="Valeur", theta="Indicateur",
                   color="Type", template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r,
                   )
fig.update_layout(
    polar = dict(
        radialaxis = dict(showticklabels=False, ticks=''),
    )
)
fig.show()

In [ ]:
fig = px.line_polar(indicateurs, r="Valeur", theta="Indicateur",
                   color="Type", template="plotly_dark",
                   line_close=True,
                   color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.update_traces(fill='toself')
fig.update_layout(
    polar = dict(
        radialaxis = dict(showticklabels=False, ticks=[-2, -1, 0, 1, 2]),
    )
)
fig.show()